In [2]:
import numpy as np
import pandas as pd

# Args

In [3]:
EXP = "exp2"
PLATFORM = "_ILLUMINA"

# Input

In [8]:
blast_annotations = "/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/" + EXP + "_MHC" + PLATFORM + "/blast/expected_templates/ILLUMINA-MHC-2.blast.tsv"

In [16]:
HEAD_FILE = "/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/" + EXP + "_MHC" + PLATFORM + "/cutadapt_naming/no_adapters/headers/annotated_headers.lst"

# Output

In [10]:
head_blast_annotations = "/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/" + EXP + "_MHC" + PLATFORM + "/blast/expected_templates/blast.annotated.tsv"

# Main

In [12]:
blast_df = pd.read_csv(blast_annotations, sep="\t", names=['query_id', 'template_id', 'identity',
                                                             'alignment_length','mismatches', 'gap_openings',
                                                             'q_alignment_start','q_alignment_end',
                                                             't_alignment_start', 't_alignment_end','e-value',
                                                             'bit_score'])

In [17]:
head_df = pd.read_csv(HEAD_FILE, sep=" ", names=['query_id', 'gem', 'tso', 'b_primer', 'anneal', 'cd8_primer', 'mhc_primer'])
head_df['a_primer'] = np.where(head_df.cd8_primer == 'no_adapter', head_df.mhc_primer, head_df.cd8_primer)
head_df.replace('no_adapter', np.NaN, inplace=True)
head_df['match'] = head_df.loc[:, ['tso', 'b_primer', 'anneal', 'a_primer']].count(axis=1)

In [18]:
head_df.head()

,query_id,gem,tso,b_primer,anneal,cd8_primer,mhc_primer,a_primer,match


In [7]:
blast_df.replace({"gem": r"^BX:Z:"}, {"gem": ""}, regex=True, inplace=True)
doublets = blast_df.duplicated(['query_id', 'gem', 'bit_score'], keep=False) #, 't_alignment_start', 't_alignment_end', 'q_alignment_start', 'q_alignment_end', 'match'
blast_df['credible_alignment'] = np.where(doublets, False, True)
blast_df.sort_values(by=['gem', 'query_id', 'bit_score', 'alignment_length'], ascending=False, inplace=True) #
blast_df.drop_duplicates(['query_id'], inplace=True)
blast_df['barcode'], blast_df['sample'] = blast_df.template_id.str.split("_", n=1).str

In [8]:
blast_df.to_csv(clean_blast_annotations, sep="\t", index=False)